# Chr17-22, GTEx Adipose (subcutaneous) fusion weights

## Part1: simulation and mr.ash

* set up

In [1]:
wkdir = "/project2/mstephens/causalTWAS/simulations/simulation_ashtest_20200721/"

In [2]:
codedir = "/project2/mstephens/causalTWAS/causal-TWAS/code/"

In [3]:
geno = "/project2/mstephens/causalTWAS/ukbiobank/ukb_chr17to22_s20000"

In [4]:
weight = "/project2/mstephens/causalTWAS/fusion_weights/Adipose_Subcutaneous"

In [5]:
maintag = '20200721-4'

In [6]:
subtags =  [str(i) for i in [2,4,5,6,8,]]

* prepare param 

In [7]:
cd {wkdir}

/project2/mstephens/causalTWAS/simulations/simulation_ashtest_20200721


In [8]:
template = "{wkdir}/param-{maintag}.R".format(wkdir = wkdir, maintag = maintag)

In [9]:
a = open(template, 'r').readlines()

In [11]:
for subtag in subtags:
    with open("param-{maintag}-{subtag}.R".format(maintag = maintag, subtag = subtag), 'w') as newpar:
        a[5] = 'SED <- {sed}\n'.format(sed = str(950 + int(subtag)))
        newpar.writelines(a)

* commands

In [12]:
g2dir = "cd {wkdir}"

In [13]:
simu = "Rscript {codedir}/run_simulate_data.R {geno} {weight} param-{tag}.R simu_{tag}"

In [14]:
simu_expr = "{wkdir}/simu_{tag}-cis-expr.Rd"

In [15]:
simu_pheno = "{wkdir}/simu_{tag}-pheno.Rd"

In [16]:
mrash2 = "Rscript {codedir}/run_test_mr.ash2s.R {geno} {simu_expr} {simu_pheno} {tag}"

In [17]:
exprgwas = "Rscript {codedir}/run_gwas_expr.R {simu_expr} {simu_pheno} {tag}"

In [18]:
snpgwas = "Rscript {codedir}/run_gwas_snp.R {geno} {simu_pheno} {tag}"

In [19]:
ld = "Rscript {codedir}/run_ld.R {geno} {simu_expr} {simu_pheno} {tag}"

* write master file

In [20]:
tags = [maintag + '-' + subtag for subtag in subtags]

In [21]:
masterfile = 'master_run4-x.sh'

In [22]:
cd {wkdir}

/project2/mstephens/causalTWAS/simulations/simulation_ashtest_20200721


In [23]:
with open(masterfile, 'w') as outf:
    for tag in tags:
        outf.write(';'.join([
              'source activate twas', 
              g2dir,
              #simu,
              mrash2,
              exprgwas,
              snpgwas,
              #ld
        ]).format(
              simu_expr = simu_expr,
              simu_pheno = simu_pheno,
              wkdir = wkdir,
              codedir = codedir,
              geno = geno,
              weight = weight,
              tag = tag
        ).format(
              wkdir = wkdir,
              tag = tag
        ) + '\n')

In [24]:
!python /home/simingz/cancer_somatic/cancer_somatic/annotation/pMake_batchjobs_RCC.py {masterfile} --mem 32 --ntasks 6 --partition mstephens

planning to run 4 jobs ...
Type: sbatch submit_parallel_master_run4-x.sh.sh


## Part 2: susie

* set up 

In [25]:
susiedir = "/project2/mstephens/causalTWAS/simulations/simulation_susietest_20200721"

In [26]:
mrashdir = wkdir
mrashres = "{mrashdir}/{tag}-mr.ash2s.{tag2}"
prior = "{mrashdir}/{tag}-mr.ash2s.{tag2}.param.txt"

* commands

In [27]:
g2susiedir = "cd {susiedir}"
susie = "Rscript {codedir}/run_test_susie.R {geno} {simu_expr} {simu_pheno} {prior} {mrashres} {tag}.{tag2} 3"

* write commands

In [28]:
cd {susiedir}

/project2/mstephens/causalTWAS/simulations/simulation_susietest_20200721


In [29]:
susietags = [maintag + '-' + str(subtag) for subtag in range(1,51)]
tag2s = ['zerose-es', 'lassoes-se', 'zeroes-es', 'lassoes-es']

In [30]:
susiemasterfile = 'susie_run1-x.sh'

In [31]:
with open(susiemasterfile, 'w') as outf:
    for tag in susietags:
        for tag2 in tag2s:
            outf.write(';'.join([
                'source activate twas', 
                g2susiedir,
                susie
            ]).format(
                susiedir = susiedir,
                codedir = codedir,
                geno = geno,
                simu_expr = simu_expr,
                simu_pheno = simu_pheno,
                prior = prior,
                mrashres = mrashres,
                tag = tag, 
                tag2 = tag2,
            ).format(
                wkdir = wkdir,
                mrashdir = mrashdir,
                tag = tag, 
                tag2 = tag2,
            ) + '\n')

In [32]:
!python /home/simingz/cancer_somatic/cancer_somatic/annotation/pMake_batchjobs_RCC.py {susiemasterfile} --mem 28 --ntasks 1 --partition broadwl

planning to run 200 jobs ...
Type: sbatch submit_parallel_susie_run1-x.sh.sh


## Part3: susie, fix prior

In [42]:
susiedir = "/project2/mstephens/causalTWAS/simulations/simulation_susietest_20200721/20200721-3-fixprior_causal"

In [43]:
mrashdir = wkdir
# mrashres = "{mrashdir}/{tag}-mr.ash2s.lassoes-es"
mrashres = "/home/simingz/causalTWAS/simulations/shared_files/20200721-1-1-mr.ash2s.lassoes-es"
prior = "{susiedir}/20200721-3-fixedprior_{partag}.txt"
susiemaintag = "20200721-3"

* commands

In [44]:
g2susiedir = "cd {susiedir}"
susie = "Rscript {codedir}/run_test_susie.R {geno} {simu_expr} {simu_pheno} {prior} {mrashres} {tag}.fixedprior{partag} 1"

* write commands

In [45]:
cd {susiedir}

/project2/mstephens/causalTWAS/simulations/simulation_susietest_20200721/20200721-3-fixprior_causal


In [46]:
susietags = [susiemaintag + '-' + str(subtag) for subtag in range(1,101)]
partags = ['1', '2', '3']

In [47]:
susiemasterfile = 'susie_run3-x-fixedp.sh'

In [48]:
with open(susiemasterfile, 'w') as outf:
    for tag in susietags:
        for partag in partags:
            outf.write(';'.join([
                'source activate twas', 
                g2susiedir,
                susie
            ]).format(
                susiedir = susiedir,
                codedir = codedir,
                geno = geno,
                simu_expr = simu_expr,
                simu_pheno = simu_pheno,
                prior = prior,
                mrashdir = mrashdir,
                mrashres = mrashres,
                tag = tag,
                partag = partag
            ).format(
                mrashdir = mrashdir,
                wkdir = wkdir,
                susiedir = susiedir,
                tag = tag,
                partag = partag
            ) + '\n')

In [51]:
!python /home/simingz/cancer_somatic/cancer_somatic/annotation/pMake_batchjobs_RCC.py {susiemasterfile} --mem 28 --ntasks 1 --partition mstephens

planning to run 300 jobs ...
Type: sbatch submit_parallel_susie_run3-x-fixedp.sh.sh


In [52]:
more susie_run3-x-fixedp.sh

source activate twas;cd /project2/mstephens/causalTWAS/simulations/simulation_susietest_20200721/20200721-3-fixprior_causal;Rscript /project2/mstephens/causalTWAS/causal-TWAS/code//run_test_susie.R /project2/mstephens/causalTWAS/ukbiobank/ukb_chr17to22_s20000 /project2/mstephens/causalTWAS/simulations/simulation_ashtest_20200721//simu_20200721-3-1-cis-expr.Rd /project2/mstephens/causalTWAS/simulations/simulation_ashtest_20200721//simu_20200721-3-1-pheno.Rd /project2/mstephens/causalTWAS/simulations/simulation_susietest_20200721/20200721-3-fixprior_causal/20200721-3-fixedprior_1.txt /home/simingz/causalTWAS/simulations/shared_files/20200721-1-1-mr.ash2s.lassoes-es 20200721-3-1.fixedprior1 1
source activate twas;cd /project2/mstephens/causalTWAS/simulations/simulation_susietest_20200721/20200721-3-fixprior_causal;Rscript /project2/mstephens/causalTWAS/causal-TWAS/code//run_test_susie.R /project2/mstephens/causalTWAS/ukbiobank/ukb_chr17to22_s20000 /project2/mstephens/causalTWAS/simulation